In [ ]:
# Importing Keras and Tensorflow modules

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import os.path
import cv2

In [ ]:
# Initilize the CNN

classifier = Sequential()

In [ ]:
# Step 1 - Convolution

classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))

In [ ]:
# Step 2 - Pooling

classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Step 2(b) - Add 2nd Convolution Layer making it Deep followed by a Pooling Layer

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Step 3 - Flattening

classifier.add(Flatten())

In [ ]:
# Step 4 - Fully Connected Neural Network

# Hidden Layer - Activation Function RELU
classifier.add(Dense(units = 128, activation = 'relu')) 
# Output Layer - Activation Function Softmax(to clasify multiple classes)
classifier.add(Dense(units = 3, activation = 'softmax'))

In [ ]:
# Compile the CNN

# Categorical Crossentropy - to classify between multiple classes of images
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Image Augmentation and Training Section

# Image Augmentation to prevent Overfitting (Applying random transformation on images to train set.ie. 
# scalling, rotating and streching)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(128, 128),
        batch_size=16,
        class_mode='categorical')

In [ ]:
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(128, 128),
        batch_size=16,
        class_mode='categorical')

In [ ]:
#Fit the clasifier on the CNN data
if(os.path.isfile('my_model.h5') == False):
    classifier.fit_generator(
            training_set,
            steps_per_epoch=8000,
            epochs=10,
            validation_data=test_set,
            validation_steps=2000
    )
    # Save the generated model to my_model.h5
    classifier.save('my_model.h5')
else:
    classifier = load_model('my_model.h5')
        